# Inception V4 Pretrained Model (timm)

`timm` 라이브러리를 사용하여 ImageNet으로 pretrain된 순수 Inception V4 모델을 불러와서 운전자 행동 분류를 수행합니다.

In [1]:
!pip install pandas numpy

  Using cached pandas-2.3.3-cp312-cp312-win_amd64.whl.metadata (19 kB)
  Using cached numpy-2.3.4-cp312-cp312-win_amd64.whl.metadata (60 kB)
  Using cached pytz-2025.2-py2.py3-none-any.whl.metadata (22 kB)
  Using cached tzdata-2025.2-py2.py3-none-any.whl.metadata (1.4 kB)
Using cached pandas-2.3.3-cp312-cp312-win_amd64.whl (11.0 MB)
Using cached numpy-2.3.4-cp312-cp312-win_amd64.whl (12.8 MB)
Using cached pytz-2025.2-py2.py3-none-any.whl (509 kB)
Using cached tzdata-2025.2-py2.py3-none-any.whl (347 kB)

   ---------------------------------------- 0/4 [pytz]
   ---------- ----------------------------- 1/4 [tzdata]
   -------------------- ------------------- 2/4 [numpy]
   -------------------- ------------------- 2/4 [numpy]
   -------------------- ------------------- 2/4 [numpy]
   -------------------- ------------------- 2/4 [numpy]
   -------------------- ------------------- 2/4 [numpy]
   -------------------- ------------------- 2/4 [numpy]
   -------------------- ------------------

In [2]:
# 필요한 패키지 설치
!pip install timm matplotlib pillow tqdm scikit-learn

  Using cached timm-1.0.22-py3-none-any.whl.metadata (63 kB)
  Using cached matplotlib-3.10.7-cp312-cp312-win_amd64.whl.metadata (11 kB)
  Using cached pillow-12.0.0-cp312-cp312-win_amd64.whl.metadata (9.0 kB)
  Using cached tqdm-4.67.1-py3-none-any.whl.metadata (57 kB)
  Using cached scikit_learn-1.7.2-cp312-cp312-win_amd64.whl.metadata (11 kB)
  Using cached torch-2.9.0-cp312-cp312-win_amd64.whl.metadata (30 kB)
  Using cached torchvision-0.24.0-cp312-cp312-win_amd64.whl.metadata (5.9 kB)
  Using cached pyyaml-6.0.3-cp312-cp312-win_amd64.whl.metadata (2.4 kB)
  Using cached huggingface_hub-1.0.1-py3-none-any.whl.metadata (13 kB)
  Using cached safetensors-0.6.2-cp38-abi3-win_amd64.whl.metadata (4.1 kB)
  Using cached contourpy-1.3.3-cp312-cp312-win_amd64.whl.metadata (5.5 kB)
  Using cached cycler-0.12.1-py3-none-any.whl.metadata (3.8 kB)
  Using cached fonttools-4.60.1-cp312-cp312-win_amd64.whl.metadata (114 kB)
  Using cached kiwisolver-1.4.9-cp312-cp312-win_amd64.whl.metadata (6.4

In [ ]:
!pip install torch torchvision --index-url https://download.pytorch.org/whl/cu128


Looking in indexes: https://download.pytorch.org/whl/cu128


In [1]:
torch.cuda.is_available()

NameError: name 'torch' is not defined

In [2]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
import timm

# GPU 사용 가능 여부 확인
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'사용 디바이스: {device}')
if torch.cuda.is_available():
    print(f'GPU: {torch.cuda.get_device_name(0)}')

사용 디바이스: cpu


## GPU 환경 진단

In [ ]:
# 데이터 경로
base_dir = r'c:\Users\USER\PycharmProjects\DeepLearning-Term-Proj'
train_dir = os.path.join(base_dir, 'data', 'imgs', 'train')
test_dir = os.path.join(base_dir, 'data', 'imgs', 'test')

# 하이퍼파라미터
img_size = 299  # Inception V4 입력 크기
batch_size = 32
num_classes = 10  # c0 ~ c9
num_epochs = 50
learning_rate = 0.001
num_workers = 0

print(f"Train directory: {train_dir}")
print(f"Test directory: {test_dir}")
print(f"이미지 크기: {img_size}x{img_size}")
print(f"배치 크기: {batch_size}")

In [ ]:
class DriverDataset(Dataset):
    """운전자 행동 데이터셋"""
    
    def __init__(self, data_dir, transform=None, is_test=False):
        self.data_dir = data_dir
        self.transform = transform
        self.is_test = is_test
        self.images = []
        self.labels = []
        
        if is_test:
            # 테스트 데이터는 test 폴더 직접 사용
            test_images_dir = os.path.join(data_dir, 'test')
            if os.path.exists(test_images_dir):
                for img_name in os.listdir(test_images_dir):
                    if img_name.endswith(('.jpg', '.png', '.jpeg')):
                        self.images.append(os.path.join(test_images_dir, img_name))
            else:
                # test 폴더가 없으면 상위 디렉토리 확인
                for img_name in os.listdir(data_dir):
                    if img_name.endswith(('.jpg', '.png', '.jpeg')):
                        self.images.append(os.path.join(data_dir, img_name))
        else:
            # 학습 데이터는 c0~c9 폴더에서 로드
            for class_idx in range(10):
                class_name = f'c{class_idx}'
                class_path = os.path.join(data_dir, class_name)
                
                if not os.path.exists(class_path):
                    print(f"경고: {class_path} 폴더를 찾을 수 없습니다.")
                    continue
                
                for img_name in os.listdir(class_path):
                    if img_name.endswith(('.jpg', '.png', '.jpeg')):
                        img_path = os.path.join(class_path, img_name)
                        self.images.append(img_path)
                        self.labels.append(class_idx)
        
        print(f"{'테스트' if is_test else '학습'} 데이터: {len(self.images)}개 이미지")
    
    def __len__(self):
        return len(self.images)
    
    def __getitem__(self, idx):
        img_path = self.images[idx]
        image = Image.open(img_path).convert('RGB')
        
        if self.transform:
            image = self.transform(image)
        
        if self.is_test:
            return image, os.path.basename(img_path)
        else:
            label = self.labels[idx]
            return image, label

In [ ]:
# Baseline w/o data augmentation ver.
# 데이터 증강 - 학습용
train_transform = transforms.Compose([
    transforms.Resize((img_size, img_size)),
    # transforms.RandomHorizontalFlip(p=0.5),
    # transforms.RandomRotation(15),
    # transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
    # transforms.RandomAffine(degrees=0, translate=(0.1, 0.1)),
    transforms.ToTensor(),
    # transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # ImageNet 정규화 -> 추후 우리 데이터셋으로 정규화 진행 예정
])

# 검증/테스트용 (증강 없음)
val_transform = transforms.Compose([
    transforms.Resize((img_size, img_size)),
    transforms.ToTensor(),
    # transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

print("데이터 변환 정의 완료")

In [ ]:
# 전체 학습 데이터 로드
full_dataset = DriverDataset(train_dir, transform=train_transform, is_test=False)

# 학습/검증 분할 (80% 학습, 20% 검증)
train_size = int(0.8 * len(full_dataset))
val_size = len(full_dataset) - train_size
train_dataset, val_dataset = torch.utils.data.random_split(full_dataset, [train_size, val_size])

# 검증 데이터는 증강 없이 사용하기 위해 transform 변경
val_dataset.dataset = DriverDataset(train_dir, transform=val_transform, is_test=False)
val_indices = val_dataset.indices
val_dataset = torch.utils.data.Subset(val_dataset.dataset, val_indices)

# 데이터 로더
train_loader = DataLoader(
    train_dataset, 
    batch_size=batch_size, 
    shuffle=True, 
    num_workers=0,  # Windows에서는 0 권장
    pin_memory=True if torch.cuda.is_available() else False
)

val_loader = DataLoader(
    val_dataset, 
    batch_size=batch_size, 
    shuffle=False, 
    num_workers=0,
    pin_memory=True if torch.cuda.is_available() else False
)

print(f"학습 샘플: {len(train_dataset)}개")
print(f"검증 샘플: {len(val_dataset)}개")
print(f"학습 배치 수: {len(train_loader)}")
print(f"검증 배치 수: {len(val_loader)}")

In [5]:
# timm에서 사용 가능한 Inception V4 모델 확인
print("사용 가능한 Inception V4 모델:")
inception_models = timm.list_models('*inception_v4*', pretrained=True)
for model_name in inception_models:
    print(f"  - {model_name}")

사용 가능한 Inception V4 모델:
  - inception_v4.tf_in1k


In [ ]:
# Inception V4 pretrained 모델 생성
print("\nInception V4 pretrained 모델 로딩 중...")
model = timm.create_model(
    'inception_v4',  # 순수 Inception V4 (Inception-ResNet-V2가 아님!)
    pretrained=True,  # ImageNet pretrained 가중치 사용
    num_classes=num_classes  # 우리 데이터셋에 맞게 출력 레이어 변경
)

model = model.to(device)
print(f"✓ Inception V4 모델이 {device}에 로드되었습니다.")

# 모델 정보 출력
num_params = sum(p.numel() for p in model.parameters())
num_trainable = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"전체 파라미터: {num_params:,}")
print(f"학습 가능 파라미터: {num_trainable:,}")

In [ ]:
# 손실 함수
criterion = nn.CrossEntropyLoss()

# 옵티마이저 (Adam)
optimizer = optim.Adam(model.parameters(), lr=learning_rate, weight_decay=1e-4)

# 학습률 스케줄러 (성능이 향상되지 않으면 학습률 감소)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(
    optimizer, 
    mode='max',  # validation accuracy를 모니터링
    factor=0.5, 
    patience=5, 
    verbose=True
)

print("손실 함수, 옵티마이저, 스케줄러 설정 완료")

In [ ]:
def train_epoch(model, dataloader, criterion, optimizer, device):
    """1 에폭 학습"""
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    
    pbar = tqdm(dataloader, desc='Training')
    for images, labels in pbar:
        images = images.to(device)
        labels = labels.to(device)
        
        # Forward
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        # Backward
        loss.backward()
        optimizer.step()
        
        # 통계
        running_loss += loss.item() * images.size(0)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        
        # 진행률 표시
        pbar.set_postfix({
            'loss': f'{loss.item():.4f}',
            'acc': f'{100. * correct / total:.2f}%'
        })
    
    epoch_loss = running_loss / total
    epoch_acc = 100. * correct / total
    return epoch_loss, epoch_acc


def validate(model, dataloader, criterion, device):
    """검증"""
    model.eval()
    running_loss = 0.0
    correct = 0
    total = 0
    
    with torch.no_grad():
        pbar = tqdm(dataloader, desc='Validation')
        for images, labels in pbar:
            images = images.to(device)
            labels = labels.to(device)
            
            outputs = model(images)
            loss = criterion(outputs, labels)
            
            running_loss += loss.item() * images.size(0)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            
            pbar.set_postfix({
                'loss': f'{loss.item():.4f}',
                'acc': f'{100. * correct / total:.2f}%'
            })
    
    epoch_loss = running_loss / total
    epoch_acc = 100. * correct / total
    return epoch_loss, epoch_acc

print("학습/검증 함수 정의 완료")

In [ ]:
# 학습 히스토리 저장
history = {
    'train_loss': [],
    'train_acc': [],
    'val_loss': [],
    'val_acc': []
}

# 최고 성능 모델 저장
best_val_acc = 0.0
best_model_path = 'best_inception_v4_model.pth'

print("=" * 70)
print(f"학습 시작: {num_epochs} 에폭")
print("=" * 70)

for epoch in range(num_epochs):
    print(f'\nEpoch {epoch+1}/{num_epochs}')
    print('-' * 70)
    
    # 학습
    train_loss, train_acc = train_epoch(model, train_loader, criterion, optimizer, device)
    
    # 검증
    val_loss, val_acc = validate(model, val_loader, criterion, device)
    
    # 히스토리 저장
    history['train_loss'].append(train_loss)
    history['train_acc'].append(train_acc)
    history['val_loss'].append(val_loss)
    history['val_acc'].append(val_acc)
    
    # 결과 출력
    print(f'\n📊 Epoch {epoch+1} 결과:')
    print(f'  Train Loss: {train_loss:.4f} | Train Acc: {train_acc:.2f}%')
    print(f'  Val Loss:   {val_loss:.4f} | Val Acc:   {val_acc:.2f}%')
    
    # 학습률 스케줄러 업데이트
    scheduler.step(val_acc)
    
    # 최고 성능 모델 저장
    if val_acc > best_val_acc:
        best_val_acc = val_acc
        torch.save({
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'val_acc': val_acc,
        }, best_model_path)
        print(f'  ✓ 최고 성능 모델 저장! (Val Acc: {val_acc:.2f}%)')

print("\n" + "=" * 70)
print("학습 완료!")
print(f"최고 검증 정확도: {best_val_acc:.2f}%")
print("=" * 70)

In [ ]:
# 학습 곡선 시각화
plt.figure(figsize=(15, 5))

# Loss 그래프
plt.subplot(1, 2, 1)
plt.plot(history['train_loss'], label='Train Loss', marker='o', linewidth=2)
plt.plot(history['val_loss'], label='Validation Loss', marker='s', linewidth=2)
plt.title('Model Loss', fontsize=14, fontweight='bold')
plt.xlabel('Epoch', fontsize=12)
plt.ylabel('Loss', fontsize=12)
plt.legend(fontsize=10)
plt.grid(True, alpha=0.3)

# Accuracy 그래프
plt.subplot(1, 2, 2)
plt.plot(history['train_acc'], label='Train Accuracy', marker='o', linewidth=2)
plt.plot(history['val_acc'], label='Validation Accuracy', marker='s', linewidth=2)
plt.title('Model Accuracy', fontsize=14, fontweight='bold')
plt.xlabel('Epoch', fontsize=12)
plt.ylabel('Accuracy (%)', fontsize=12)
plt.legend(fontsize=10)
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('training_history.png', dpi=300, bbox_inches='tight')
plt.show()

print(f"최고 Train Accuracy: {max(history['train_acc']):.2f}%")
print(f"최고 Validation Accuracy: {max(history['val_acc']):.2f}%")
print(f"최종 Train Accuracy: {history['train_acc'][-1]:.2f}%")
print(f"최종 Validation Accuracy: {history['val_acc'][-1]:.2f}%")

In [ ]:
# 최고 성능 모델 로드
checkpoint = torch.load(best_model_path)
model.load_state_dict(checkpoint['model_state_dict'])
print(f"✓ 최고 성능 모델 로드 완료 (Epoch {checkpoint['epoch']+1}, Val Acc: {checkpoint['val_acc']:.2f}%)")

In [ ]:
# 테스트 데이터셋 및 로더 생성
test_dataset = DriverDataset(test_dir, transform=val_transform, is_test=True)
test_loader = DataLoader(
    test_dataset, 
    batch_size=batch_size, 
    shuffle=False, 
    num_workers=0
)

print(f"테스트 샘플 수: {len(test_dataset)}개")
print(f"테스트 배치 수: {len(test_loader)}")

In [ ]:
# 테스트 데이터 예측
print("\n테스트 데이터 예측 중...")
model.eval()

predictions = []
img_names = []

with torch.no_grad():
    pbar = tqdm(test_loader, desc='Predicting')
    for images, filenames in pbar:
        images = images.to(device)
        
        # 예측
        outputs = model(images)
        probs = torch.softmax(outputs, dim=1)  # 확률값으로 변환
        
        predictions.append(probs.cpu().numpy())
        img_names.extend(filenames)

# 예측 결과 합치기
predictions = np.vstack(predictions)
print(f"✓ 예측 완료: {len(img_names)}개 이미지")
print(f"예측 shape: {predictions.shape}")

In [ ]:
# Submission 파일 생성
submission = pd.DataFrame({
    'img': img_names,
    'c0': predictions[:, 0],
    'c1': predictions[:, 1],
    'c2': predictions[:, 2],
    'c3': predictions[:, 3],
    'c4': predictions[:, 4],
    'c5': predictions[:, 5],
    'c6': predictions[:, 6],
    'c7': predictions[:, 7],
    'c8': predictions[:, 8],
    'c9': predictions[:, 9]
})

# CSV 파일 저장
submission_file = 'inception_v4_submission.csv'
submission.to_csv(submission_file, index=False)

print("\n" + "=" * 70)
print(f"✓ Submission 파일 생성 완료: {submission_file}")
print(f"✓ 총 {len(submission)}개 이미지 예측")
print("=" * 70)

# 샘플 출력
print("\n📋 Submission 파일 샘플:")
print(submission.head(10))

# 통계
print("\n📊 예측 통계:")
for i in range(10):
    col_name = f'c{i}'
    print(f"  {col_name}: mean={submission[col_name].mean():.4f}, "
          f"std={submission[col_name].std():.4f}, "
          f"max={submission[col_name].max():.4f}")

In [ ]:
# 샘플 이미지와 예측 결과 시각화
def show_predictions(num_samples=8):
    """샘플 이미지와 예측 결과 시각화"""
    
    # 랜덤 샘플 선택
    indices = np.random.choice(len(test_dataset), num_samples, replace=False)
    
    fig, axes = plt.subplots(2, 4, figsize=(16, 8))
    axes = axes.ravel()
    
    for idx, sample_idx in enumerate(indices):
        # 이미지 로드
        img_path = test_dataset.images[sample_idx]
        img = Image.open(img_path).convert('RGB')
        
        # 예측 결과 가져오기
        img_name = os.path.basename(img_path)
        pred_idx = img_names.index(img_name)
        pred_probs = predictions[pred_idx]
        pred_class = np.argmax(pred_probs)
        pred_conf = pred_probs[pred_class]
        
        # 시각화
        axes[idx].imshow(img)
        axes[idx].axis('off')
        axes[idx].set_title(
            f'Predicted: c{pred_class}\nConfidence: {pred_conf:.2%}',
            fontsize=10,
            fontweight='bold'
        )
    
    plt.tight_layout()
    plt.savefig('sample_predictions.png', dpi=300, bbox_inches='tight')
    plt.show()

show_predictions(8)
print("✓ 샘플 예측 결과 시각화 완료")

In [ ]:
# 최종 모델 저장
final_model_path = 'inception_v4_final.pth'
torch.save({
    'model_state_dict': model.state_dict(),
    'history': history,
    'best_val_acc': best_val_acc,
}, final_model_path)

print(f"✓ 최종 모델 저장 완료: {final_model_path}")

# 모델 로드 방법
print("\n📌 모델 로드 방법:")
print(f"""
# 모델 불러오기
import timm
import torch

model = timm.create_model('inception_v4', pretrained=False, num_classes={num_classes})
checkpoint = torch.load('{final_model_path}')
model.load_state_dict(checkpoint['model_state_dict'])
model.eval()
""")

## 완료! 🎉

- ✅ timm에서 순수 **Inception V4** pretrained 모델 사용
- ✅ ImageNet 가중치로 전이 학습
- ✅ 데이터 증강 및 학습/검증 분할
- ✅ 학습 결과 시각화
- ✅ 테스트 데이터 예측
- ✅ **Submission 파일 생성** (`inception_v4_submission.csv`)
- ✅ 모델 저장 및 로드 방법

생성된 파일:
- `best_inception_v4_model.pth` - 최고 성능 모델
- `inception_v4_final.pth` - 최종 모델
- `inception_v4_submission.csv` - 제출 파일
- `training_history.png` - 학습 곡선 그래프
- `sample_predictions.png` - 샘플 예측 결과